<a href="https://colab.research.google.com/github/ekity1002/PyOptBook-memo/blob/main/Python%E3%81%A7%E3%81%AF%E3%81%98%E3%82%81%E3%82%8B%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96_6%E7%AB%A0%E3%83%A1%E3%83%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/ohmsha/PyOptBook.git
! cd PyOptBook; pip install -r requirements.txt

fatal: destination path 'PyOptBook' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# サークルにおける学生の乗車グループ分け問題
 条件の整理
 * 学生をどの車に割り当てるかを決定する
 * 乗車人数は定員を超えてはいけない
 * 運転免許を持つ学生を最低一人は車に割り当てる
 * 各学年の学生を一人以上車に割り当てる
 * 男女一人以上を車に割り当てる

In [3]:
import pandas as pd
students_df = pd.read_csv('/content/PyOptBook/6.api/resource/students.csv')
cars_df = pd.read_csv('/content/PyOptBook/6.api/resource/cars.csv')

In [7]:
print(students_df.shape)
students_df

(24, 4)


,student_id,license,gender,grade
0,0,0,0,1
1,1,1,1,2
2,2,1,0,3
3,3,1,1,4
4,4,0,1,1
5,5,0,1,2
6,6,0,0,3
7,7,0,1,4
8,8,1,1,1
9,9,1,0,2


In [6]:
cars_df

,car_id,capacity
0,0,6
1,1,6
2,2,5
3,3,4
4,4,5
5,5,5


# モデル構築

In [19]:
import pulp

# 学生の乗車グループ分け問題(0-1整数計画問題)のインスタンス
prob = pulp.LpProblem('cars', pulp.LpMinimize)

#学生のリスト
S = students_df['student_id'].tolist()
# 車のリスト
C = cars_df['car_id'].tolist()
# 学年のリスト
G = [1,2,3,4]
# 学生の車のペアのリスト
SC = [(s,c) for s in S for c in C]
# 免許を持っている学生のリスト
S_license = students_df[students_df['license'] == 1]['student_id']
# 学年が g の学生のリスト
S_g = {g: students_df[students_df['grade'] == g]['student_id'] for g in G}
#男性と女性のリスト
S_male = students_df[students_df['gender']==0]['student_id']
S_female = students_df[students_df['gender']==1]['student_id']

In [20]:
# 車の乗車定員の定数
U = cars_df['capacity'].to_list()

#変数
# 学生をどの車に割り当てるかの変数
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

# 制約
# １：各学生を１つの車に割り当てる
for s in S:
    prob += pulp.lpSum([x[s,c] for c in C]) == 1

# 2:各車には定員より多く乗ることはできない
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S]) <= U[c]

# 3:各車にはドライバーを一人以上割り当てる
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_license]) >= 1

# 4: 各車に各学年の学生を一人以上割り当てる
for c in C:
    for g in G:
        #print(c,g)
        prob += pulp.lpSum([x[s,c] for s in S_g[g]]) >= 1

# 5.1: 各車に男性を一人以上割り当てる
for c in C:
    prob += pulp.lpSum(x[s,c] for s in S_male) >= 1

# 5.2: 各車に女性を一人以上割り当てる
for c in C:
    prob += pulp.lpSum(x[s,c] for s in S_female) >= 1

In [22]:
# 球界
status = prob.solve()
print('Status', pulp.LpStatus[status])

Status Optimal


In [26]:
#最適化結果の表示
car2students= {c: [s for s in S if x[s,c].value() == 1] for c in C}

# 車の乗車定員
max_people = dict(zip(cars_df['car_id'], cars_df['capacity']))

In [28]:
for c, ss in car2students.items():
    print(f"車{c}, 学生数: {len(ss)}, 定員; {max_people[c]}")
    print(f"学生: {ss}")

車0, 学生数: 4, 定員; 6
学生: [6, 7, 8, 9]
車1, 学生数: 4, 定員; 6
学生: [4, 18, 21, 23]
車2, 学生数: 4, 定員; 5
学生: [5, 19, 20, 22]
車3, 学生数: 4, 定員; 4
学生: [3, 10, 13, 16]
車4, 学生数: 4, 定員; 5
学生: [0, 2, 11, 17]
車5, 学生数: 4, 定員; 5
学生: [1, 12, 14, 15]
